### Examples Of Pyspark ML

#### INICIAMOS SESION

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('dataframe').getOrCreate()

In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/tables/"))

path,name,size,modificationTime
dbfs:/FileStore/tables/test1-1.csv,test1-1.csv,143,1671399222000
dbfs:/FileStore/tables/test1.csv,test1.csv,143,1671399119000


In [0]:
training = spark.read.csv("dbfs:/FileStore/tables/test1.csv", inferSchema=True, header=True)

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

CARGAMOS CSV

training = spark.read.csv('data/test1.csv', header=True, inferSchema=True)

In [0]:
training = spark.read.csv('data/test1.csv', header=True, inferSchema=True)

In [0]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



En PySpark se trabaja de forma diferente. Tendremos que agrupar nuestras variables independientes de forma que queden todas en una columna y dentro de una lista, por lo que crearemos un vector de ensamblaje o "vector assembler", de tal modo que queden así esas variables independientes:
- [Age, Experience]

Lo que haremos con estas dos, será tratarlas como una nueva variable independiente:
- [Age, Experience] ----> nueva_variable_independiente

In [0]:
from pyspark.ml.feature import VectorAssembler 

In [0]:
feature_assembler = VectorAssembler(inputCols=['age', 'Experience'], outputCol='Independent features') # 1:46:27 del vídeo

In [0]:
output = feature_assembler.transform(training)

Veremos que se crea una nueva columna cuyos valores se corresponden a unos array con el contenido de aquellas variables independientes que hemos agrupado. Esto será nuestro input feature o lo que solíamos definir como train.

In [0]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



Seleccionamos las columnas que nos interesan para nuestro modelo: el train (Independent Features) y el test (Salary)

In [0]:
finalized_data = output.select('Independent features', 'Salary')
finalized_data.show()

+--------------------+------+
|Independent features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



A continuación, entrenaremos un modelo de regresión lineal.

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
(train, test) = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent features', labelCol='Salary')
regressor = regressor.fit(train)

In [0]:
regressor.coefficients

Out[33]: DenseVector([-90.5483, 1608.7819])

In [0]:
regressor.intercept

Out[34]: 16079.136690647425

In [0]:
prediction = regressor.evaluate(test)

In [0]:
prediction.predictions.show()

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
+--------------------+------+-----------------+
|Independent features|Salary|       prediction|
+--------------------+------+-----------------+
|          [23.0,2.0]| 18000|17214.09079632846|
+--------------------+------+-----------------+



In [0]:
# Errores

prediction.meanAbsoluteError, prediction.meanSquaredError

Out[37]: (785.909203671541, 617653.2764156357)